# Make Shapes and Create their Persistence Diagrams

## Load packages

In [13]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
path = '../../scripts/'
sys.path.insert(0,path)
from RipserToDict import ripser_to_dict
from PlotPersistence import plot_persistence
import pickle as pickle

## Parameters

In [2]:
# general params
params = {
    'all_seeds': range(0,5000),
    'dim': 2,
    'max_persistence': 2.5,
    'n_points': 200,
    'threshold': 10
}

In [3]:
# circle params
circle_params = params.copy()
circle_params['deviation'] = 0.1
circle_params['dim'] = 1

In [4]:
# sphere params
sphere_params = params.copy()
sphere_params['deviation'] = 0.1

In [5]:
# torus params
torus_params = params.copy()
torus_params['deviation'] = [0.1,0.1]
print(torus_params)

{'n_points': 200, 'dim': 2, 'max_persistence': 2.5, 'deviation': [0.1, 0.1], 'all_seeds': range(0, 5000), 'threshold': 10}


## Functions

In [6]:
def create_circle_distance_matrix(params):
    n_points = params['n_points']
    dev = params['deviation']
    seed = params['seed']
    
    # construct unit circle, r=1±dev, 0<theta<2*pi
    np.random.seed(seed=seed)
    circular_coords = np.random.rand(2,n_points)
    circular_coords[0,:] = 1. - dev/2. + dev*circular_coords[0,:]
    circular_coords[1,:] = 2*np.pi*circular_coords[1,:]
    
    # transform into Euclidean
    Euclidean_coords = np.zeros((2,n_points))
    Euclidean_coords[0,:] = circular_coords[0,:]*np.cos(circular_coords[1,:])
    Euclidean_coords[1,:] = circular_coords[1,:]*np.sin(circular_coords[1,:])
    
    # calculate distances
    distance_matrix = np.zeros((n_points,n_points))
    for i in range(n_points):
        for j in range(n_points):
            distance_matrix[i,j] = np.sqrt(
            (Euclidean_coords[0,i] - Euclidean_coords[0,j])**2
            + (Euclidean_coords[1,i] - Euclidean_coords[1,j])**2)
            
    params['shape'] = 'circle'
    return distance_matrix

In [7]:
def create_sphere_distance_matrix(params):
    n_points = params['n_points']
    error = params['deviation']
    seed = params['seed']
    
    # construct (unit) sphere in spherical coordinates: r=1±dev, 0<theta<pi, 0<phi<2*pi
    np.random.seed(seed=seed)
    noisy_spherical_coords = np.random.rand(3,n_points)
    noisy_spherical_coords[0,:] = 1. - error/2. + error*noisy_spherical_coords[0,:]
    noisy_spherical_coords[1,:] = noisy_spherical_coords[1,:]*np.pi
    noisy_spherical_coords[2,:] = noisy_spherical_coords[2,:]*2*np.pi

    # transform perfect sphere into Euclidean coordinates
    noisy_Euclidean_coords = np.zeros((3,n_points))
    noisy_Euclidean_coords[0,:] = noisy_spherical_coords[0,:]*np.sin(noisy_spherical_coords[1,:])*np.cos(noisy_spherical_coords[2,:])
    noisy_Euclidean_coords[1,:] = noisy_spherical_coords[0,:]*np.sin(noisy_spherical_coords[1,:])*np.sin(noisy_spherical_coords[2,:])
    noisy_Euclidean_coords[2,:] = noisy_spherical_coords[0,:]*np.cos(noisy_spherical_coords[1,:])

    # create distance matrix for the perfect sphere
    noisy_pairwise_distances = np.zeros((n_points,n_points))
    for i in range(n_points):
        for j in range(n_points):
            noisy_pairwise_distances[i,j] = np.sqrt(
                (noisy_Euclidean_coords[0,j] - noisy_Euclidean_coords[0,i])**2
                + (noisy_Euclidean_coords[1,j] - noisy_Euclidean_coords[1,i])**2
                + (noisy_Euclidean_coords[2,j] - noisy_Euclidean_coords[2,i])**2
            )
    
    params['shape'] = 'sphere'
    return noisy_pairwise_distances

In [8]:
def create_torus_distance_matrix(params):
    n_points = params['n_points']
    seed = params['seed']
    deviation = params['deviation']
    
    # create noise
    a = 1. - deviation[0]/2. + np.random.rand(1,n_points)
    b = 1. - deviation[1]/2. + np.random.rand(1,n_points)
    
    # create random points
    np.random.seed(seed=seed)
    random_points = 2*np.pi*np.random.rand(2,n_points)
    R4_coords = np.zeros((4,n_points))
    R4_coords[0,:] = a*np.cos(random_points[0,:])
    R4_coords[1,:] = a*np.sin(random_points[0,:])
    R4_coords[2,:] = b*np.cos(random_points[1,:])
    R4_coords[3,:] = b*np.sin(random_points[1,:])
    
    # calculate distances
    distance_matrix = np.zeros((n_points,n_points))
    for i in range(n_points):
        for j in range(n_points):
            distance_matrix[i,j] = np.sqrt(
            (R4_coords[0,i] - R4_coords[0,j])**2
            + (R4_coords[1,i] - R4_coords[1,j])**2
            + (R4_coords[2,i] - R4_coords[2,j])**2
            + (R4_coords[3,i] - R4_coords[3,j])**2)
            
    params['shape'] = 'torus'
    return distance_matrix

## Calculations

In [9]:
# circles
for seed in circle_params['all_seeds']:
    
    circle_params['seed'] = seed
    circle_distance_matrix = create_circle_distance_matrix(circle_params)
    np.savetxt("temp/circle_distance.txt", circle_distance_matrix, delimiter=",")
    
    !cd ../../../ripser; \
    ./ripser --format distance --dim {circle_params['dim']} --threshold {circle_params['threshold']} \
    ../topology_immersion_Henrik/exercises/ml_on_1_to_3/temp/circle_distance.txt \
    > ../topology_immersion_Henrik/exercises/ml_on_1_to_3/temp/circle_persistence.txt
    
    circle_persistence = ripser_to_dict('temp/circle_persistence.txt',params).copy()
    circle_persistence['params'] = circle_params
    
    with open('../../../heavy_files/exercises/ml_on_1_to_3/circle_persistences/' + str(seed) + '.txt', "wb") as myFile:
        pickle.dump(circle_persistence, myFile)

In [10]:
# spheres
for seed in sphere_params['all_seeds']:
    
    sphere_params['seed'] = seed
    sphere_distance_matrix = create_sphere_distance_matrix(sphere_params)
    np.savetxt("temp/sphere_distance.txt", sphere_distance_matrix, delimiter=",")
    
    !cd ../../../ripser; \
    ./ripser --format distance --dim {sphere_params['dim']} --threshold {sphere_params['threshold']} \
    ../topology_immersion_Henrik/exercises/ml_on_1_to_3/temp/sphere_distance.txt \
    > ../topology_immersion_Henrik/exercises/ml_on_1_to_3/temp/sphere_persistence.txt
    
    sphere_persistence = ripser_to_dict('temp/sphere_persistence.txt',params).copy()
    sphere_persistence['params'] = sphere_params
    
    with open('../../../heavy_files/exercises/ml_on_1_to_3/sphere_persistences/' + str(seed) + '.txt', "wb") as myFile:
        pickle.dump(sphere_persistence, myFile)

In [11]:
# toruses
for seed in torus_params['all_seeds']:

    torus_params['seed'] = seed
    torus_distance_matrix = create_torus_distance_matrix(torus_params)
    np.savetxt("temp/torus_distance.txt", torus_distance_matrix, delimiter=",")
    
    !cd ../../../ripser; \
    ./ripser --format distance --dim {torus_params['dim']} --threshold {torus_params['threshold']} \
    ../topology_immersion_Henrik/exercises/ml_on_1_to_3/temp/torus_distance.txt \
    > ../topology_immersion_Henrik/exercises/ml_on_1_to_3/temp/torus_persistence.txt
    
    torus_persistence = ripser_to_dict('temp/torus_persistence.txt',params).copy()
    torus_persistence['params'] = torus_params
    
    with open('../../../heavy_files/exercises/ml_on_1_to_3/torus_persistences/' + str(seed) + '.txt', "wb") as myFile:
        pickle.dump(torus_persistence, myFile)
    

max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small
max_persistence is too small


## Henceforth, nothing of value

SyntaxError: invalid syntax (<ipython-input-12-724cd97c378d>, line 1)